In [33]:
import sys
sys.path.append('/opt/conda/pkgs')
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4262b1bf4bf1ffb403c0eb7a42ad5906_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4506eccf78279d93d0e8a34c035e91c5_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/6bda807e3967eae797c7b1b9eeaee8db_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c2a47d89d1d34e789fdf782557bb7194_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c6c5514ada15b890fb27d1e36371554c_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/d964a294c2d0fef56a434c021026281e_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/e1c932db5cd4271709e54d8028824bc9_/lib/python3.12/site-packages")

import gzip, json
from Bio import SeqIO
import scipy as sp
from collections import Counter
import numpy as np
import pandas as pd
import pickle, os, gzip, json, sys, itertools
from pathlib import Path
from importlib import reload
from dataclasses import dataclass, field
import collections
import matplotlib.pyplot as plt
import networkx as nx
import pysam
import seaborn
import sharedmem
import pygraphviz
import  sklearn

In [19]:
sys.path.append("scripts")
sys.path.append("../../scripts")
from data_io import is_fwd_id, get_fwd_id, get_sibling_id
from graph import OverlapGraph, GenomicInterval, get_overlap_statistics, remove_false_edges
from truth import get_overlaps
from evaluate import NearestNeighborsConfig, mp_compute_nearest_neighbors
from plots import plot_read_graph, mp_plot_read_graphs, get_graphviz_layout, get_umap_layout
from nearest_neighbors import (
    ExactNearestNeighbors,
    NNDescent,
    WeightedLowHash,
    PAFNearestNeighbors,
    LowHash,
    HNSW,
    ProductQuantization,
    _NearestNeighbors,
    IVFProductQuantization,
)

In [10]:
MAX_SAMPLE_SIZE = int(1e9)
COVERAGE_DEPTH = 20
max_n_neighbors = 20

npz_path = "/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/CHM13/IGK/pbsim_ONT_93_30k/kmer_k16/feature_matrix.npz"
tsv_path = "/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/CHM13/IGK/pbsim_ONT_93_30k//kmer_k16/metadata.tsv.gz"
json_path = "/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/CHM13/IGK/pbsim_ONT_93_30k/kmer_k16/read_features.json.gz"

meta_df = pd.read_table(tsv_path).iloc[:MAX_SAMPLE_SIZE, :].reset_index()
read_indices = {read_name: read_id for read_id, read_name in meta_df['read_name'].items()}
feature_matrix = sp.sparse.load_npz(npz_path)[meta_df.index, :]

with gzip.open(json_path, "rt") as f:
    read_features = json.load(f)
    read_features = {i: read_features[i] for i in meta_df.index}

In [11]:
def get_read_intervals(meta_df):
    read_intervals = {
        i: [GenomicInterval(strand, start, end)]
        for i, strand, start, end in zip(
            meta_df.index,
            meta_df["reference_strand"],
            meta_df["reference_start"],
            meta_df["reference_end"],
        )
    }
    return read_intervals

read_intervals = get_read_intervals(meta_df)

reference_graph = OverlapGraph.from_intervals(read_intervals)
nr_edges = set((node_1, node_2) for node_1, node_2, data in reference_graph.edges(data=True) if not data['redundant'])
connected_component_count = len(list(nx.connected_components(reference_graph)))
len(reference_graph.nodes), len(reference_graph.edges), len(nr_edges), connected_component_count



(5106, 103266, 9866, 2)

In [ ]:
##dimension reduction


In [ ]:
config = NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,None,None',
            tfidf='None',
            nearest_neighbors_kw=dict(metric='euclidean')) 
processes = 8
configs = [config]

nbr_dict, time_dict, memory_dict = mp_compute_nearest_neighbors(
    data=feature_matrix,
    configs=configs,
    n_neighbors=max_n_neighbors,
    processes=processes,
)
nbr_indices = nbr_dict[0]

In [55]:
processes = 8
read_ids = np.array(list(read_features))
graphs = collections.defaultdict(dict)
k_values = np.arange(2, max_n_neighbors + 1)
def from_matrix_to_evaluation(feature_matrix):
    config = NearestNeighborsConfig(
            nearest_neighbors_method=ExactNearestNeighbors,
            description='ExactNearestNeighbors,Euclidean,None,None',
            tfidf='None',
            nearest_neighbors_kw=dict(metric='euclidean')) 

    configs = [config]
    nbr_dict, time_dict, memory_dict = mp_compute_nearest_neighbors(
        data=feature_matrix,
        configs=configs,
        n_neighbors=max_n_neighbors,
        processes=processes,
    )
    nbr_indices = nbr_dict[0]
    df_rows = []
    for k in k_values:
        graph = OverlapGraph.from_neighbor_indices(
            neighbor_indices=nbr_indices,
            n_neighbors=k,
            read_ids=read_ids,
            require_mutual_neighbors=False,
        )
        graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
        stats = { "n_neighbors": k}
        stats = {"description":'Exact_Euclidean', "n_neighbors": k,
                    **graph_stats}
        df_rows.append(stats)
    df = pd.DataFrame(df_rows)
    return df, nbr_indices



In [56]:
## no dimension reduction
df,nbr_indices = from_matrix_to_evaluation(feature_matrix)

Evaluating 1 configs using 8 processes.
0 Nearest neighbors.
Finished NearestNeighborsConfig(description='ExactNearestNeighbors,Euclidean,None,None', tfidf='None', dimension_reduction_method=None, nearest_neighbors_method=<class 'nearest_neighbors.ExactNearestNeighbors'>). Elapsed time: {'nearest_neighbors': 4.480342864990234}. Peak memory: {}



In [62]:
nbr_indices2

array([[   0, 2503, 3032, ..., 3011, 3379,  875],
       [   1, 2502, 3033, ..., 3010, 3378,  874],
       [   2, 3305, 1946, ..., 4226, 2938,  177],
       ...,
       [5103, 3455,  308, ..., 3959, 2783, 2903],
       [5104, 1827,  703, ..., 3751, 2156, 4220],
       [5105, 1826,  702, ..., 3750, 2157, 4221]])

In [57]:
nbr_indices

array([[   0, 2235,  431, ..., 3032, 2024, 1453],
       [   1, 2234,  430, ..., 3033, 2025, 1452],
       [   2,  450, 4765, ..., 2487, 2587, 1072],
       ...,
       [5103, 4812, 2740, ..., 5000,  796, 3450],
       [5104,  855,  854, ..., 1905, 3303, 3302],
       [5105,  854,  855, ..., 1904, 3303, 3302]])

In [42]:
df

,description,n_neighbors,precision,nr_precision,recall,nr_recall,singleton_count,singleton_fraction,N50,N50_percentage
0,Exact_Euclidean,2,0.671766,0.250919,0.054858,0.214474,1380,0.270270,17,0.33%
1,Exact_Euclidean,3,0.644535,0.222594,0.078399,0.283398,896,0.175480,47,0.92%
2,Exact_Euclidean,4,0.615938,0.197424,0.099549,0.333975,612,0.119859,116,2.27%
3,Exact_Euclidean,5,0.591209,0.179307,0.119575,0.379586,457,0.089503,118,2.31%
4,Exact_Euclidean,6,0.567460,0.163422,0.137461,0.414352,374,0.073247,122,2.39%
5,Exact_Euclidean,7,0.547518,0.152976,0.154649,0.452260,305,0.059734,202,3.96%
6,Exact_Euclidean,8,0.528266,0.141946,0.170753,0.480235,260,0.050920,216,4.23%
7,Exact_Euclidean,9,0.509678,0.133082,0.185879,0.508007,224,0.043870,478,9.36%
8,Exact_Euclidean,10,0.493899,0.124738,0.200686,0.530509,198,0.038778,478,9.36%
9,Exact_Euclidean,11,0.478238,0.117798,0.214301,0.552504,174,0.034078,478,9.36%


In [45]:
processes = 8
read_ids = np.array(list(read_features))
graphs = collections.defaultdict(dict)
k_values = np.arange(2, max_n_neighbors + 1)
def from_nbr_to_evaluation(nbr_indices):
    df_rows = []
    for k in k_values:
        graph = OverlapGraph.from_neighbor_indices(
            neighbor_indices=nbr_indices,
            n_neighbors=k,
            read_ids=read_ids,
            require_mutual_neighbors=False,
        )
        graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
        stats = { "n_neighbors": k}
        stats = {"description":'Exact_Euclidean', "n_neighbors": k,
                    **graph_stats}
        df_rows.append(stats)
    df = pd.DataFrame(df_rows)
    return df

In [30]:
feature_matrix.shape

(5106, 656109)

In [60]:
nbrs = sklearn.neighbors.NearestNeighbors(
    n_neighbors=20,metric='cosine'
)
nbrs.fit(feature_matrix)
_, nbr_indices2 = nbrs.kneighbors(feature_matrix)
df = from_matrix_to_evaluation(nbr_indices)
df

Evaluating 1 configs using 8 processes.
0 Nearest neighbors.
Finished NearestNeighborsConfig(description='ExactNearestNeighbors,Euclidean,None,None', tfidf='None', dimension_reduction_method=None, nearest_neighbors_method=<class 'nearest_neighbors.ExactNearestNeighbors'>). Elapsed time: {'nearest_neighbors': 0.3513045310974121}. Peak memory: {}



(        description  n_neighbors  precision  nr_precision    recall  \
 0   Exact_Euclidean            2   0.010940      0.003878  0.000765   
 1   Exact_Euclidean            3   0.011476      0.003976  0.001230   
 2   Exact_Euclidean            4   0.011392      0.003753  0.001675   
 3   Exact_Euclidean            5   0.011765      0.003624  0.002169   
 4   Exact_Euclidean            6   0.011779      0.003551  0.002634   
 5   Exact_Euclidean            7   0.011566      0.003448  0.003021   
 6   Exact_Euclidean            8   0.011225      0.003120  0.003380   
 7   Exact_Euclidean            9   0.011135      0.002962  0.003786   
 8   Exact_Euclidean           10   0.011109      0.002758  0.004212   
 9   Exact_Euclidean           11   0.010923      0.002574  0.004561   
 10  Exact_Euclidean           12   0.010606      0.002376  0.004842   
 11  Exact_Euclidean           13   0.010502      0.002269  0.005200   
 12  Exact_Euclidean           14   0.010397      0.002155  0.00

In [47]:
nbr_indices

array([[   0, 2503, 3032, ..., 3011, 3379,  875],
       [   1, 2502, 3033, ..., 3010, 3378,  874],
       [   2, 3305, 1946, ..., 4226, 2938,  177],
       ...,
       [5103, 3455,  308, ..., 3959, 2783, 2903],
       [5104, 1827,  703, ..., 3751, 2156, 4220],
       [5105, 1826,  702, ..., 3750, 2157, 4221]])

In [51]:

nbrs = sklearn.neighbors.NearestNeighbors(
    n_neighbors=20,  metric='minkowski',p=2,algorithm='ball_tree'
)
nbrs.fit(feature_matrix)
_, nbr_indices = nbrs.kneighbors(feature_matrix)
df =from_matrix_to_evaluation(nbr_indices)
df

Evaluating 1 configs using 8 processes.
0 Nearest neighbors.
Finished NearestNeighborsConfig(description='ExactNearestNeighbors,Euclidean,None,None', tfidf='None', dimension_reduction_method=None, nearest_neighbors_method=<class 'nearest_neighbors.ExactNearestNeighbors'>). Elapsed time: {'nearest_neighbors': 0.33452343940734863}. Peak memory: {}



,description,n_neighbors,precision,nr_precision,recall,nr_recall,singleton_count,singleton_fraction,N50,N50_percentage
0,Exact_Euclidean,2,0.016158,0.004637,0.001114,0.003345,5101,0.999021,1,0.02%
1,Exact_Euclidean,3,0.017933,0.004908,0.001840,0.005271,5088,0.996475,1,0.02%
2,Exact_Euclidean,4,0.017685,0.004508,0.002469,0.006588,5076,0.994125,1,0.02%
3,Exact_Euclidean,5,0.017488,0.004386,0.003012,0.007906,5064,0.991774,1,0.02%
4,Exact_Euclidean,6,0.017310,0.004095,0.003602,0.008920,5043,0.987662,1,0.02%
5,Exact_Euclidean,7,0.016994,0.003857,0.004096,0.009730,5028,0.984724,1,0.02%
6,Exact_Euclidean,8,0.016816,0.003854,0.004648,0.011149,5005,0.980219,1,0.02%
7,Exact_Euclidean,9,0.016502,0.003813,0.005113,0.012366,4988,0.976890,1,0.02%
8,Exact_Euclidean,10,0.016037,0.003679,0.005529,0.013278,4973,0.973952,1,0.02%
9,Exact_Euclidean,11,0.015734,0.003588,0.005946,0.014190,4959,0.971210,1,0.02%


In [39]:
feature_matrix

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 9353746 stored elements and shape (5106, 656109)>